In [ ]:
import glob
import sys
sys.path.append('.')

import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

import conversions as c

plt.style.use('./style.mplstyle')

Template notebook to give examples of how to preprocess data, product plots, etc.

Example dataset used here is from NJ Motorsports park

# Load data

In [ ]:
# Downsampling the data significantly speeds up plotting times,
# and improves appearance of the plots, but is not strictly critical for analysis
DOWNSAMPLE_FACTOR = 3 # keep every Nth row
DOWNSAMPLE = True

dfs = {}
for idx, file in enumerate(glob.glob('./data/session*.csv')):
    dfs[idx] = pd.read_csv(file)
    dfs[idx] = dfs[idx].dropna(subset=['lap_number'])
    if DOWNSAMPLE:
        dfs[idx] = dfs[idx][::DOWNSAMPLE_FACTOR]
    print(file)

In [ ]:
# convert kpa to psi, C to F
for i in dfs:
    dfs[i]['engine_oil_pressure_psi'] = dfs[i].apply(lambda x: c.kpa_to_psi(x['engine_oil_pressure']), axis=1)
    dfs[i]['fuel_pressure_psi'] = dfs[i].apply(lambda x: c.kpa_to_psi(x['fuel_pressure']), axis=1)
    dfs[i]['engine_oil_temp_f'] = dfs[i].apply(lambda x: c.c_to_f(x['engine_oil_temp']), axis=1)

# Basic line plots

Basic temporal plots for visualization.

In [ ]:
x = 'elapsed_time'
y1 = 'engine_oil_pressure_psi'
y2 = 'lateral_acc'

nrows = len(dfs)
fig, ax = plt.subplots(nrows, 1, figsize=(15,20))
axs = ax.flatten()
for i in dfs:
    a = axs[i]
    # create a dual axis plot if desired
    a2 = a.twinx()
    sns.lineplot(ax = a, x = x, y = y1, data = dfs[i], lw = 1, color='darkorange', label = y1, legend=False)
    sns.lineplot(ax = a2, x = x, y = y2, data = dfs[i], lw = 1, color='navy', label = y2, legend=False)
    a.set_title(f'session {i}', fontsize=14)
    a.set_ylim([0,100])
    a.legend(loc=(0.7,0.9))
    a2.legend(loc=(0.7,0.8))
fig.subplots_adjust(hspace=0.5)
fig.savefig(f'./figures/session-time_x_op-latacc.png', bbox_inches='tight', facecolor='w', dpi=150)

# Correlations with rpm, oil temp

It is well established when oil pressure drops occur.
Some readings display right hand turns as negative Gs while others appear as positive Gs so it is always good to check.

Since oil pressure is also dependent on engine speed, it is good to factor in engine speed when examining low oil pressures

In [ ]:
session = 3

test_df = dfs[session]#.query("rpm>5000")

In [ ]:
fig, ax = plt.subplots(figsize=(7,5))
sns.scatterplot(ax = ax, x = 'lateral_acc', y = 'engine_oil_pressure_psi', hue='rpm', data = test_df,
                palette='rocket')
ax.set_xlim([-6,6])
ax.set_title('Lateral Accel vs Oil Pressure')
fig.savefig(f'./figures/session_{session}_lat-acc_vs_oil-pressure.png', bbox_inches='tight', facecolor='w')
pass

In [ ]:
fig, ax = plt.subplots(figsize=(7,5))
sns.scatterplot(ax = ax, x = 'engine_oil_temp_f', y = 'engine_oil_pressure_psi', hue='rpm', data = test_df,
                palette='rocket')
ax.set_title('Oil Temp vs Oil Pressure')
fig.savefig(f'./figures/session_{session}_oil-temp_vs_oil-pressure.png', bbox_inches='tight', facecolor='w')
pass

# Plotting coordinates

In [ ]:
# convert to geodataframes
gdfs = {}
for i in dfs:
    gdfs[i] = gpd.GeoDataFrame(data = dfs[i],
                               geometry = gpd.points_from_xy(dfs[i]['longitude'], dfs[i]['latitude']))

## use fastest lap

In [ ]:
def lap_times(df: pd.DataFrame,
              lap_num_col: str = 'lap_number',
              time_col: str = 'elapsed_time') -> dict:

    return (df.groupby(lap_num_col)[time_col].max() - df.groupby(lap_num_col)[time_col].min()).to_dict()

In [ ]:
fastest_lap = {}
for i in gdfs:
    lt = lap_times(gdfs[i], lap_num_col='lap_number', time_col='elapsed_time')
    fl = min(lt, key=lt.get)
    text = f"Session {i}, fastest lap: {fl} time: {lt[fl]:.3f}"
    fastest_lap[i] = fl
    print(text)

In [ ]:
col = 'engine_oil_pressure_psi'

fig, ax = plt.subplots(2,3, figsize=(15,10))
axs = ax.flatten()
for i in gdfs:
    a = axs[i]
    fl = fastest_lap[i]
    data = gdfs[i].loc[gdfs[i]['lap_number'] == fl]
    data.plot(ax = a, column = col, legend=True, vmin=40, vmax=70)
    a.set_title(f'Session {i}')
    a.axis('off')
for j in range(i+1, 2*3):
    fig.delaxes(axs[j])
fig.subplots_adjust(hspace=0.3)
fig.savefig(f'./figures/gps_{col}.png', bbox_inches='tight', facecolor='w', dpi=150)